# Zero to Hero

## 1.12 Funcion de Ganancia de Arbol training/testing

El objetivo es poner el notebook anterior dentro de una funcion, e invocarla con distintas semillas, para maravillarnos ante la dispersion de las ganancias

In [2]:
options(warn=-1)

In [3]:
rm(list=ls())
gc()

library( "data.table")   #cargo la libreria  data.table
library( "rpart")  #cargo la libreria  rpart
library( "rpart.plot")
library("caret")

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,1941868,103.8,3720028,198.7,3720028,198.7
Vcells,3242558,24.8,8388608,64.0,5252768,40.1


In [5]:
options(repr.plot.width = 20, repr.plot.height = 10) 
setwd("C:\\Users\\oliva\\OneDrive\\Desktop\\DS\\Austral\\08 - Labo1") #Aqui se debe poner la ruta de la PC local

In [7]:
GananciaArbol  <-  function(semilla, data, x, train=0.70) {
  #establezco la semilla
  set.seed(semilla)
  train_rows <- createDataPartition(dataset$clase_ternaria, p = 0.70, list = FALSE)
  
  modelo <-  rpart(formula = "clase_ternaria ~ ." ,
                          data = data[ train_rows],  #los datos de training
                          xval = 0, #no cross validation
                          control = x)
    
  #Aplico el modelo a los datos de testing  
  prediccion  <- predict(modelo, data[-train_rows], type = "prob")
  
  prob_baja2  <- prediccion[, "BAJA+2"]
  ganancia_testing <- data[-train_rows, sum((prob_baja2>0.025) * ifelse(clase_ternaria == "BAJA+2", 59000, -1000))]
  
  return(ganancia_testing)
}

Vale la pena notar que dentro de la funcion no se han creado  dtrain y dtest,  sino que directamente se utilizan:
* data[ train_rows]  como training
* data[ -train_rows]  como testing


In [22]:
#Otra funcion con cross validation

GananciaArbol2 <- function(seed, df, x){   
#Semilla
    set.seed(seed)
    train_rows <- createDataPartition(dataset$clase_ternaria, 
                                      p = 0.8, list = FALSE)
#Modelo   
    model <- rpart("formula = clase_ternaria ~ .",
                   data = df[train_rows],
                   xval = 5, #5 cross validation,
                   control = x
                  )
#Prediccion
    prediccion <- predict(model, df[-train_rows], type = "prob"
                         )
    prob_baja2  <- prediccion[, "BAJA+2"]
    ganancia_testing <- df[-train_rows, sum((prob_baja2>0.025) * ifelse(clase_ternaria == "BAJA+2", 59000, -1000))]
  
return(ganancia_testing)
}


### Aqui empieza el programa

In [18]:
dataset <- fread("./labo/datasets/paquete_premium_202011.csv")   #cargo el dataset

In [19]:
#defino unos buenos hiperparametros
param  <- list("cp" = -0.5,
               "minsplit" =  900,
               "minbucket" = 440,
               "maxdepth" = 5
              )

#### Ahora hago algunas llamadas a la funcion

In [11]:
GananciaArbol(11, dataset, x = param)

[1] 2799000

In [12]:
GananciaArbol(13, dataset, x = param)

[1] 3520000

In [13]:
GananciaArbol(17, dataset, x = param)

[1] 2933000

In [14]:
GananciaArbol(19, dataset, x = param)

[1] 3144000

In [15]:
GananciaArbol(23, dataset, x = param)

[1] 3339000

Lo primero que se obseva es la gran variabilidad de la ganancia sin normalizar segun la semilla, hasta un 50%

| semilla | ganancia |
| --- | ---: |
| 11 | 2799000 |
| 13 | 3520000 |
| 17 | 2933000 |
| 19 | 3144000 |
| 23 | 3339000 |



Normalizando las ganancias de testing, es decir haciendo el cociente por 0.3 da lo siguiente

| semilla | ganancia |
| --- | ---: |
| 11 | 9330000|
| 13 | 11733333 |
| 17 | 9776667 |
| 19 | 10480000 |
| 23 | 11130000 |

Esta dispersión es **NOTABLE**  ya que el algoritmo es el mismo, con los mismos parámetros.
<br>
Lo único que cambia es que datos se utilizan para entrenar y testear, pero siempre es 70% / 30%
<br>
Es más notable aún que se ha tenido cuidado que la partición sea estratificada segun el campo **clase_ternaria**, lo que apriori uno supondria que va a generar particiones muy homogéneas.

Aun con CV se mantiene una alta dispersion

In [23]:
GananciaArbol2(11, dataset, x = param)

[1] 2192000

In [24]:
GananciaArbol2(13, dataset, x = param)

[1] 1545000